In [235]:
# Import libraries
from time import time
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# Algorithms
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
# Neural Networks
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
# Measures
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Measures
from sklearn import metrics

# Utils
from sklearn.preprocessing import LabelEncoder  

In [236]:
# Defining FIFA related functions

def get_fifa_stats(match, player_stats):
    ''' Aggregates fifa stats for a given match. '''    
    
    #Define variables
    match_id =  match.match_api_id
    date = match['date']
    players = ['home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
               "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
               "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
               "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
               "away_player_10", "away_player_11"]
    player_stats_new = pd.DataFrame()
    names = []
    
    #Loop through all players
    for player in players:   
            
        #Get player ID
        player_id = match[player]
        
        #Get player stats 
        stats = player_stats[player_stats.player_api_id == player_id]
            
        #Identify current stats       
        current_stats = stats[stats.date < date].sort_values(by = 'date', ascending = False)[:1]
        
        if np.isnan(player_id) == True:
            overall_rating = pd.Series(0)
        else:
            current_stats.reset_index(inplace = True, drop = True)
            overall_rating = pd.Series(current_stats.loc[0, "overall_rating"])

        #Rename stat
        name = "{}_overall_rating".format(player)
        names.append(name)
            
        #Aggregate stats
        player_stats_new = pd.concat([player_stats_new, overall_rating], axis = 1)
    
    player_stats_new.columns = names        
    player_stats_new['match_api_id'] = match_id

    player_stats_new.reset_index(inplace = True, drop = True)
    
    #Return player stats    
    return player_stats_new.iloc[0]    


def get_fifa_data(matches, player_stats, path = None, data_exists = False):
    ''' Gets fifa data for all matches. '''  
    
    #Check if fifa data already exists
    if data_exists == True:
        
        fifa_data = pd.read_pickle(path)
        
    else:
        
        print("Collecting fifa data for each match...")       
        start = time()
        
        #Apply get_fifa_stats for each match
        fifa_data = matches.apply(lambda x :get_fifa_stats(x, player_stats), axis = 1)
        
        end = time()    
        print("Fifa data collected in {:.1f} minutes".format((end - start)/60))
    
    #Return fifa_data
    return fifa_data


def get_overall_fifa_rankings(fifa, get_overall = False):
    ''' Get overall fifa rankings from fifa data. '''
      
    temp_data = fifa
    
    #Check if only overall player stats are desired
    if get_overall == True:
        
        #Get overall stats
        data = temp_data.loc[:,(fifa.columns.str.contains('overall_rating'))]
        data.loc[:,'match_api_id'] = temp_data.loc[:,'match_api_id']
    else:
        
        #Get all stats except for stat date
        cols = fifa.loc[:,(fifa.columns.str.contains('date_stat'))]
        temp_data = fifa.drop(cols.columns, axis = 1)        
        data = temp_data
    
    #Return data
    return data

In [295]:
# Defining matches related functions

''' Derives a label for a given match. '''
def get_match_outcome(match):
    
    #Define variables
    home_goals = match['home_team_goal']
    away_goals = match['away_team_goal']
     
    outcome = pd.DataFrame()
    outcome.loc[0,'match_api_id'] = match['match_api_id'] 

    #Identify match outcome  
    if home_goals > away_goals:
        outcome.loc[0,'outcome'] = "Win"
    if home_goals == away_goals:
        outcome.loc[0,'outcome'] = "Draw"
    if home_goals < away_goals:
        outcome.loc[0,'outcome'] = "Defeat"

    #Return outcome        
    return outcome.loc[0]


''' Get the last x matches of a given team. '''
def get_last_matches(matches, date, team, x = 10):
    #Filter team matches from matches
    team_matches = matches[(matches['home_team_api_id'] == team) | (matches['away_team_api_id'] == team)]
                           
    #Filter x last matches from team matches
    last_matches = team_matches[team_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:x,:]
    
    #Return last matches
    return last_matches
    
    
''' Get the last x matches of two given teams. '''
def get_last_matches_against_eachother(matches, date, home_team, away_team, x = 10):
    #Find matches of both teams
    home_matches = matches[(matches['home_team_api_id'] == home_team) & (matches['away_team_api_id'] == away_team)]    
    away_matches = matches[(matches['home_team_api_id'] == away_team) & (matches['away_team_api_id'] == home_team)]  
    total_matches = pd.concat([home_matches, away_matches])
    
    #Get last x matches
    try:    
        last_matches = total_matches[total_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:x,:]
    except:
        last_matches = total_matches[total_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:total_matches.shape[0],:]
        
        #Check for error in data
        if(last_matches.shape[0] > x):
            print("Error in obtaining matches")
            
    #Return data
    return last_matches


''' Get the goals[home & away] of a specfic team from a set of matches. '''
def get_goals(matches, team):
    home_goals = int(matches.home_team_goal[matches.home_team_api_id == team].sum())
    away_goals = int(matches.away_team_goal[matches.away_team_api_id == team].sum())

    total_goals = home_goals + away_goals
    
    return total_goals


''' Get the goals[home & away] conceided of a specfic team from a set of matches. '''
def get_goals_conceided(matches, team):
    home_goals = int(matches.home_team_goal[matches.away_team_api_id == team].sum())
    away_goals = int(matches.away_team_goal[matches.home_team_api_id == team].sum())

    total_goals = home_goals + away_goals

    return total_goals


''' Get the number of wins of a specfic team from a set of matches. '''
def get_wins(matches, team):
    #Find home and away wins
    home_wins = int(matches.home_team_goal[(matches.home_team_api_id == team) & (matches.home_team_goal > matches.away_team_goal)].count())
    away_wins = int(matches.away_team_goal[(matches.away_team_api_id == team) & (matches.away_team_goal > matches.home_team_goal)].count())

    total_wins = home_wins + away_wins

    return total_wins 


''' Create match specific features for a given match. '''
def get_match_features(match, matches, teams_stats, x = 10):
    #Define variables
    date = match.date
    home_team = match.home_team_api_id
    away_team = match.away_team_api_id
    
     # Gets home and away team_stats
    home_team_stats = teams_stats.loc[teams_stats['team_api_id'] == home_team]
    away_team_stats = teams_stats.loc[teams_stats['team_api_id'] == away_team]
    
    #Get last x matches of home and away team
    matches_home_team = get_last_matches(matches, date, home_team, x = 10)
    matches_away_team = get_last_matches(matches, date, away_team, x = 10)
    
    #Get last x matches of both teams against each other
    last_matches_against = get_last_matches_against_eachother(matches, date, home_team, away_team, x = 3)
    
    #Create goal variables
    home_goals = get_goals(matches_home_team, home_team)
    away_goals = get_goals(matches_away_team, away_team)
    home_goals_conceided = get_goals_conceided(matches_home_team, home_team)
    away_goals_conceided = get_goals_conceided(matches_away_team, away_team)
    
    #Define result data frame
    result = pd.DataFrame()
    
    #Define ID features
    result.loc[0, 'match_api_id'] = match.match_api_id
    result.loc[0, 'league_id'] = match.league_id
    
    #Create match features and team stats
    result.loc[0, 'home_team_buildUpPlaySpeed'] = home_team_stats.loc[0, 'buildUpPlaySpeed']
    result.loc[0, 'home_team_buildUpPlayPassing'] = home_team_stats.buildUpPlayPassing
    result.loc[0, 'home_team_chanceCreationPassing'] = home_team_stats.chanceCreationPassing
    result.loc[0, 'home_team_chanceCreationCrossing'] = home_team_stats.chanceCreationCrossing
    result.loc[0, 'home_team_chanceCreationShooting'] = home_team_stats.chanceCreationShooting
    result.loc[0, 'home_team_defencePressure'] = home_team_stats.defencePressure
    result.loc[0, 'home_team_defenceAggression'] = home_team_stats.defenceAggression
    result.loc[0, 'home_team_defenceTeamWidth'] = home_team_stats.defenceTeamWidth
    
    result.loc[0, 'away_team_buildUpPlaySpeed'] = away_team_stats.buildUpPlaySpeed
    result.loc[0, 'away_team_buildUpPlayPassing'] = away_team_stats.buildUpPlayPassing
    result.loc[0, 'away_team_chanceCreationPassing'] = away_team_stats.chanceCreationPassing
    result.loc[0, 'away_team_chanceCreationCrossing'] = away_team_stats.chanceCreationCrossing
    result.loc[0, 'away_team_chanceCreationShooting'] = away_team_stats.chanceCreationShooting
    result.loc[0, 'away_team_defencePressure'] = away_team_stats.defencePressure
    result.loc[0, 'away_team_defenceAggression'] = away_team_stats.defenceAggression
    result.loc[0, 'away_team_defenceTeamWidth'] = away_team_stats.defenceTeamWidth
    
    result.loc[0, 'home_team_goals_difference'] = home_goals - home_goals_conceided
    result.loc[0, 'away_team_goals_difference'] = away_goals - away_goals_conceided
    result.loc[0, 'games_won_home_team'] = get_wins(matches_home_team, home_team) 
    result.loc[0, 'games_won_away_team'] = get_wins(matches_away_team, away_team)
    result.loc[0, 'games_against_won'] = get_wins(last_matches_against, home_team)
    result.loc[0, 'games_against_lost'] = get_wins(last_matches_against, away_team)
    
    #Return match features
    return result.loc[0]

''' Create and aggregate features and labels for all matches. '''
def get_features(matches, teams_stats, fifa, x = 10, get_overall = False):  
    #Get fifa stats features
    fifa_stats = get_overall_fifa_rankings(fifa, get_overall)
    
    #Get match features for all matches
    match_stats = matches.apply(lambda i: get_match_features(i, matches, teams_stats, x = 10), axis = 1)
    
    #Create dummies for league ID feature
    dummies = pd.get_dummies(match_stats['league_id']).rename(columns = lambda x: 'League_' + str(x))
    match_stats = pd.concat([match_stats, dummies], axis = 1)
    match_stats.drop(['league_id'], inplace = True, axis = 1)
    
    #Create match outcomes
    outcomes = matches.apply(get_match_outcome, axis = 1)

    #Merges features and outcomes into one frame
    features = pd.merge(match_stats, fifa_stats, on = 'match_api_id', how = 'left')
    features = pd.merge(features, outcomes, on = 'match_api_id', how = 'left')
    
    #Drop NA values
    features.dropna(inplace = True)
    
    #Return preprocessed data
    return features

In [254]:
teams

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


In [261]:
# Read database data
with sqlite3.connect("../dataset/database.sqlite") as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    matches = pd.read_sql_query("SELECT * from Match order by date desc limit 3200", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    team_attributes = pd.read_sql_query("SELECT distinct * from Team_Attributes group by team_api_id order by date desc",con)
    player = pd.read_sql_query("SELECT * from Player",con)
    player_attributes = pd.read_sql_query("SELECT * from Player_Attributes",con)
    
# player stats
#cards = pd.read_csv("../dataset/card_detail.csv")
#crosses = pd.read_csv("../dataset/cross_detail.csv")
# goals = pd.read_csv("../dataset/goal_detail.csv")
#shots_off = pd.read_csv("../dataset/shotoff_detail.csv")
#shots_on = pd.read_csv("../dataset/shoton_detail.csv")
#fouls_commited = pd.read_csv("../dataset/foulcommit_detail.csv")
# team stats
#possessions = pd.read_csv("../dataset/possession_detail.csv")
#corners = pd.read_csv("../dataset/corner_detail.csv")

In [264]:
# clean team attributes
teams_stats = pd.merge(teams, team_attributes, on = 'team_api_id', how='left')
teams_stats.drop(['buildUpPlaySpeedClass', 'buildUpPlayDribblingClass', 'buildUpPlayPassingClass', 'buildUpPlayPositioningClass',
            'chanceCreationPassingClass', 'chanceCreationCrossingClass',  'chanceCreationShootingClass',
            'chanceCreationPositioningClass', 'defencePressureClass', 'defenceAggressionClass', 'defenceTeamWidthClass',
            'defenceDefenderLineClass'], inplace = True, axis = 1)
teams_stats.dropna(subset = ['team_fifa_api_id_x'], inplace = True)

In [265]:
# cleaning matches
matches.drop(['SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'], inplace = True, axis = 1)
matches = matches[matches['home_team_api_id'].isin(teams_stats['team_api_id'].tolist())]
matches = matches[matches['away_team_api_id'].isin(teams_stats['team_api_id'].tolist())]

# Reduce match data
rows = ['country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'match_api_id', 'home_team_api_id', 
        'away_team_api_id', 'home_team_goal', 'away_team_goal', "home_player_1", "home_player_2",
        "home_player_3", "home_player_4", "home_player_5", "home_player_6", "home_player_7", 
        "home_player_8", "home_player_9", "home_player_10", "home_player_11", "away_player_1",
        "away_player_2", "away_player_3", "away_player_4", "away_player_5", "away_player_6",
       "away_player_7", "away_player_8", "away_player_9", "away_player_10", "away_player_11"]

matches.dropna(subset = rows, inplace = True)

In [266]:
fifa_data = get_fifa_data(matches, player_attributes, None, data_exists = False)

Fifa data collected in 6.7 minutes


In [296]:
# Creates features and labels based on the provided data
viables = get_features(small_matches, teams_stats, fifa_data, 10, False)
inputs = viables.drop('match_api_id', axis=1)
outcomes = inputs.loc[:, 'outcomes']

KeyError: 0

In [260]:
# all features except outcomes
features = inputs.drop('outcomes', axis=1)
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(features, outcomes, test_size = 0.2, shuffle = True, stratify = outcomes)

KeyError: "['outcomes'] not found in axis"

In [ ]:
# Training and Evaluating Models
def train_model(clf, X_train, y_train):
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    print ('Training took {} seconds'.format(end - start))
    
def predict_outcome(clf, X_test, y_test):
    start = time()
    y_predict = clf.predict(X_test)
    end = time()
    print ('Predictions took {} seconds'.format(end - start))
    print("Accuracy score for training set: ", metrics.accuracy_score(y_test, y_predict))
    
def train_predict(clf, X_train, y_train, X_test, y_test):
    train_model(clf, X_train, y_train)
    predict_outcome(clf, X_test, y_test)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5)
train_predict(clf, X_train, y_train, X_test, y_test)

In [ ]:
clf = DecisionTreeClassifier(random_state=0)
train_predict(clf, X_train, y_train, X_test, y_test)

In [ ]:
clf = svm.SVC()
train_predict(clf, X_train, y_train, X_test, y_test)

In [ ]:
clf = Sequential()
clf.add(Dense(16, input_shape=(30,), activation='relu'))
clf.add(Dense(8, activation='relu'))
clf.add(Dense(4, activation='relu'))
clf.add(Dense(1, activation='sigmoid'))
clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(clf.summary())

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
print(y_train)

In [ ]:
clf.fit(X_train, y_train, epochs=150, batch_size=10)
accuracy = clf.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
from keras.utils import to_categorical
X_train = to_categorical(X_train)

In [ ]:
X_train.head()